In [1]:
%config Completer.use_jedi = False
%matplotlib inline

In [2]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display()

In [3]:
# 读取数据集
# 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式
# 并除以255使得所有像素的数值均在0到1之间
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True,transform=trans,download=True)
mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False,transform=trans, download=True)

In [6]:
len(mnist_train), len(mnist_test)

(60000, 10000)

In [9]:
"""
Fashion-MNIST中包含的10个类别分别为t-shirt（T恤）、trouser（裤子）、pullover（套衫）、dress（连衣裙）
coat（外套）、sandal（凉鞋）、shirt（衬衫）、sneaker（运动鞋）、bag（包）和ankle boot（短靴）。
以下函数用于在数字标签索引及其文本名称之间进行转换。
"""
def get_fashion_mnist_labels(labels): 
    """返回Fashion-MNIST数据集的文本标签。"""
    text_labels = [
        't-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt',
        'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [8]:
# 创建一个函数来可视化这些样本。
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.1):
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [18]:
# 以下是训练数据集中前几个样本的图像及其相应的标签（文本形式）
X, y = next(iter(data.DataLoader(mnist_train,shuffle=True, batch_size=6)))
#show_images(X.reshape(6, 28, 28), 2, 3, titles=get_fashion_mnist_labels(y));
y

tensor([5, 8, 4, 2, 0, 9])

In [19]:
# 读取小批量
batch_size = 256

def get_dataloader_workers():  #@save
    """使用4个进程来读取数据。"""
    return 4

train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True,num_workers=4)

In [20]:
# 看一下读取数据的时间
timer = d2l.Timer()
for X, y in train_iter:
    continue
f'{timer.stop():.2f} sec'

'6.74 sec'

In [28]:
 # 完整的读取例子
def load_data_fashion_mnist(batch_size, resize=None): 
    """下载Fashion-MNIST数据集，然后将其加载到内存中。"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # 训练集
    train_dataset = torchvision.datasets.FashionMNIST(root="../data",train=True,transform=trans,download=True)
    train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True,num_workers=get_dataloader_workers())
    # 测试集
    test_dataset = torchvision.datasets.FashionMNIST(root="../data",train=False,transform=trans,download=True)
    test_dataloader = data.DataLoader(test_dataset, batch_size, shuffle=False,num_workers=get_dataloader_workers())
    return (train_dataloader,test_dataloader)

In [30]:
train_iter, test_iter = load_data_fashion_mnist(6, resize=None)
for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break

torch.Size([6, 1, 28, 28]) torch.float32 torch.Size([6]) torch.int64


In [31]:
y

tensor([1, 1, 4, 5, 3, 5])